In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
train=pd.read_csv('/content/gdrive/My Drive/ML data sets /analytics Vidhya/Loan prediction/train_ctrUa4K.csv')
test=pd.read_csv('/content/gdrive/My Drive/ML data sets /analytics Vidhya/Loan prediction/test_lAUu6dG.xls')

In [4]:
df = pd.concat([train, test])

In [5]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
#replaces nan values with mode of 
df['Gender'].fillna('Male',inplace=True)

In [7]:
#replaces nan values with mode of Self_Employed
df['Self_Employed'].fillna('No',inplace=True)

In [8]:
df['Credit_History'].mode()

0    1.0
dtype: float64

In [9]:
df['Loan_Amount_Term'].fillna('342.0',inplace=True)
df['Dependents'].fillna('0',inplace=True)
df['Married'].fillna('Yes',inplace=True)

In [10]:
df['LoanAmount'].fillna('142.0',inplace=True)

In [11]:
df['Credit_History'].fillna('1.0',inplace=True)

In [12]:
df['Dependents'].replace({'3+':3},inplace=True)

In [13]:
df.describe(include='all')

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
count,981,981,981,981,981,981,981.000000,981.000000,981.0,981.0,981.0,981,614
unique,981,2,2,4,2,2,NaN,NaN,233.0,13.0,3.0,3,2
top,LP002950,Male,Yes,0,Graduate,No,NaN,NaN,120.0,360.0,1.0,Semiurban,Y
freq,1,799,634,570,763,862,NaN,NaN,29.0,823.0,754.0,349,422
mean,NaN,NaN,NaN,NaN,NaN,NaN,5179.795107,1601.916330,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,5695.104533,2718.772806,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2875.000000,0.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,3800.000000,1110.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,5516.000000,2365.000000,NaN,NaN,NaN,NaN,NaN


In [14]:
pd.DataFrame((df.isna().sum()/len(df)).sort_values(ascending=False), columns=['% NaNs'])

,% NaNs
Loan_Status,0.374108
Property_Area,0.000000
Credit_History,0.000000
Loan_Amount_Term,0.000000
LoanAmount,0.000000
CoapplicantIncome,0.000000
ApplicantIncome,0.000000
Self_Employed,0.000000
Education,0.000000
Dependents,0.000000


In [15]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,142.0,360,1,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,Y


In [16]:
df['Credit_History'].replace({'1.0':1.0},inplace=True)

In [17]:
dummy_list=['Gender', 'Married', 'Dependents', 'Education','Self_Employed','Credit_History', 'Property_Area',]

In [18]:
dummy_drop = []
clean_df = df
for i in dummy_list:
    dummy_drop += [i + '_' + str(df[i].unique()[-1])]

df = pd.get_dummies(df, columns = dummy_list)
df = df.drop(dummy_drop, axis = 1)

In [19]:
df['Loan_Status'].replace({'Y':1,'N':0},inplace=True)


In [20]:
df.head()

,Loan_ID,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Loan_Status,Gender_Male,Married_No,Dependents_0,Dependents_1,Dependents_2,Education_Graduate,Self_Employed_No,Credit_History_1.0,Property_Area_Rural,Property_Area_Urban
0,LP001002,5849,0.0,142.0,360,1.0,1,1,1,0,0,1,1,1,0,1
1,LP001003,4583,1508.0,128,360,0.0,1,0,0,1,0,1,1,1,1,0
2,LP001005,3000,0.0,66,360,1.0,1,0,1,0,0,1,0,1,0,1
3,LP001006,2583,2358.0,120,360,1.0,1,0,1,0,0,0,1,1,0,1
4,LP001008,6000,0.0,141,360,1.0,1,1,1,0,0,1,1,1,0,1


In [21]:
train_len = len(train)
train_ = df[:train_len]
test_ = df[train_len:]

In [22]:
from sklearn.model_selection import train_test_split,cross_val_score

In [23]:
y=train_['Loan_Status'].values.reshape(-1,1)
x=train_.drop(['Loan_Status','Loan_ID'],axis=1)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [24]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 600, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 300, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100,1000]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10, 15, 20, 25]

In [ ]:
# Create the random grid
random_grid_01 = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid_01)

{'n_estimators': [100, 145, 190, 236, 281, 327, 372, 418, 463, 509, 554, 600], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 64, 123, 182, 241, 300], 'min_samples_split': [2, 5, 10, 15, 100, 1000], 'min_samples_leaf': [1, 2, 5, 10, 15, 20, 25]}


In [25]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_01 = RandomForestClassifier()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random_01 = RandomizedSearchCV(estimator = rf_01, param_distributions = random_grid_01,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random_01.fit(X_train,y_train.ravel())

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   33.9s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [ ]:
rf_random_01.best_params_

{'max_depth': 182,
 'max_features': 'sqrt',
 'min_samples_leaf': 10,
 'min_samples_split': 10,
 'n_estimators': 281}

In [ ]:
rf_random_01.best_score_

-0.18540507111935683

In [ ]:
rf_random_01.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=182, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=281,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
rf_01 = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=182, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=281,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression ()

In [ ]:
cnb_model = CategoricalNB()

In [27]:
rf_01.fit(X_train, y_train.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=182, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=281,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
y_pred = rf_01.predict(X_test)

In [30]:
from sklearn import metrics

In [31]:
print('Accuracy score: ',metrics.accuracy_score(y_test, y_pred))
print('confusion matric: ',metrics.confusion_matrix(y_test, y_pred))
print('precision score: ',metrics.precision_score(y_test, y_pred))
print('precision recall curve: ',metrics.precision_recall_curve(y_test, y_pred))

Accuracy score:  0.7886178861788617
confusion matric:  [[18 25]
 [ 1 79]]
precision score:  0.7596153846153846
precision recall curve:  (array([0.6504065 , 0.75961538, 1.        ]), array([1.    , 0.9875, 0.    ]), array([0., 1.]))
